In [9]:
import glob
import pickle

import scipy.io
import scipy.signal
import librosa
import numpy as np

In [10]:
files = glob.glob('/home/dante_gates/music/Music/**/.wav', recursive=True) \
      + glob.glob('/home/dante_gates/music/Music/**/*mp3', recursive=True)

In [11]:
print(len(files))
files[:100]

11758


['/home/dante_gates/music/Music/Aesop Rock - All Day Nike + Original Run (Continuous Mix).mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI42.mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI35.mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI25.mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI17.mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI55.mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI41.mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI53.mp3',
 '/home/dante_gates/music/Music/R.C. Sproul/Foundations - An Overview of Systematic Theology/FOU01MI24.mp3',
 '/home/dante_gates/music/Music/

In [7]:
WINDOW_LENGTH = 3  # in seconds
DESIRED_SAMPLE_RATE = 44100
HOP_LENGTH = 2048
N_CLIPS = 20
OUTPUT_DIR = '/home/dante_gates/repos/music-rec/data/train'
MAXSIZE = 10  # in megabytes

In [19]:
import io
import random
import os
import pydub


def mp3_hook(f):
    audio_segment = pydub.AudioSegment.from_mp3(f)
    byte_stream = io.BytesIO()
    audio_segment.export(byte_stream, 'wav')
    byte_stream.seek(0)
    return byte_stream

def read_wav(f):
    """Read wav file and return the sample rate and audio."""
    sr, audio = scipy.io.wavfile.read(f)
    return _process_audio(sr, audio)

def _process_audio(sr, audio):
    if sr < DESIRED_SAMPLE_RATE:
        raise ValueError('audio has a sample rate less than %s'
                         % DESIRED_SAMPLE_RATE)
    return sr, audio

# TODO: not doing anything with right channel now.
def make_features(sr, audio):
    n_samples = audio.shape[0]
    try:
        L, R = audio[:, 0], audio[:, 1]
    except IndexError:
        raise ValueError('not a stereo file')
    window = sr * WINDOW_LENGTH
    samples = random.sample(list(range(n_samples - window)), k=N_CLIPS)
    melspecs = []
    for clip_begin in samples:
        clip = L[clip_begin:clip_begin+window]
        melspec = librosa.feature.melspectrogram(clip, sr, hop_length=HOP_LENGTH)
        melspecs.append(np.reshape(melspec, -1))
    return melspecs

_megabyte = 2**20
def getsize(f):
    """Return size of `f` in megabytes."""
    return os.path.getsize(f) / _megabyte

_expected_shape = (8320,)  # for sanity check
def create_training_data(files):
    for file in files:
        print('\r', 'processing', file, end='')
        if getsize(file) > MAXSIZE:
            print('\rskipping %s, file is too big' % file, end='')
            continue
        f = mp3_hook(file) if file.endswith('.mp3') else file
        try:
            sr, audio = read_wav(f)
            features = make_features(sr, audio)
        except ValueError as e:
            print(file, e)
        else:
            basename = os.path.basename(file)
            for i, feature in enumerate(features, start=1):
                if feature.shape == _expected_shape:
                    saveto = '%s - sample %s.npy' % (basename, i)
                    saveto = os.path.join(OUTPUT_DIR, saveto)
                    np.save(saveto, feature)
                else:
                    print('%s did not have expected shape' % file)

In [18]:
create_training_data(files)

 processing /home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saives/KoreyAndI/Missed Opportunity.mp3 did not have expected shape
/home/dante_gates/music/Music/Save S. & Saive

ValueError: not a stereo file